# **LAB FOUR: RELAXATION AND CONTRAST**

This lab covers...

Background

- Reintroduction of net magnetization vector
- T1 relaxation
- T2 relaxation
- intro to contrast imaging 

TR

- Background: introduce TR
- Phantoms: Phantom with three segments - two different T1 value components and an empty segment
- Exp: 2D RARE exp that measures the signal in three regions and plots the signal from these regions on the dot plot as TR is changed. This will show the noise and relaxation curves for the two materials and empty space. 
- Questions: What TR value maximises CNR? How are SNR and experiment time affected?

TE

- Background: introduce TE
- Phantoms: Phantom with three segments - two different T2 value components and an empty segment
- Exp: 2D RARE exp that measures the signal in three regions and plots the signal from these regions on the dot plot as TE is changed. This will show the noise and relaxation curves for the two materials and empty space. 
- Questions: What TE value maximises CNR? How are SNR and experiment time affected? 

Contrast Imaging

- Background: How imaging is used to highlight specific regions based on known T1 and T2 values. Introduce the concept of proton, T1 and T2 weighted images and have table for TR and TE selection based on T1, T2 values. 
- Phantoms: Contrast image with shapes that only show up on either T1 or T2 weighted images 
- Exp: 2D RARE experiment with adjustable TR and TE. Different image weighting based on table of T1,T2 values. 
- Questions: optimise parameters to maximise contrast of each hidden shape and record TR/TE values and resulting images.

References:
- https://mriquestions.com/image-contrast-trte.html
- https://mriquestions.com/fse-parameters.html

Notes:
- May need stronger spoiler gradients for short TR

>-------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
LAB_USER_NAME = 'REPLACE_ME'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [ ]:
import panel as pn
import sys
import os
import yaml
import numpy as np

from matipo import Sequence, SEQUENCE_DIR, GLOBALS_DIR, DATA_DIR
from matipo.experiment import BaseExperiment # load before pn.extension() for stylesheet changes on panel < 1.0
from matipo.experiment.plot import SignalPlot, SpectrumPlot

pn.extension()

LAB_DIR = os.path.join(DATA_DIR, LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('Data will be saved to', LAB_DIR)

## 1. Background
Since the mechanisms of relaxation occur on the quantum level, it is important to recap how individual spins contribute to the net magnetization vector. 

Taking an averaged sum of all the quantum spins' magnetic moments ( $\vec\mu$) results in a vector that describes the net direction and magnitude of these spins - this is called the Net Magnetization Vector ($\vec M$). When placed in the magnetic field ($\vec B_0$), not all the spins will align with the field, but at thermal equilibrium their net value ($\vec M$) will align. This causes a precession around $\vec B_0$ as described in Lab One. When we add energy to the system, using the oscillating magnetic field $\vec B_1$, the tip angle of $\vec M$ can be changed. 

IMAGE: Directly after a 90 pulse Mz is close to 0 and Mxy is at its maximum. 

**Relaxation** is the process of $\vec M$ returning back to its thermal equilibrium state in alignment with $\vec B_0$.

In order to better express the different types of relaxation, $\vec M$ is split into two components: the longitudinal component ($\vec M_z$) and the transverse component ($\vec M_{xy}$). Each of these components undergo relaxation at different rates and through different mechanisms.

### 1.1. Longitudinal (T1) Relaxation
Longitudinal relaxation is caused by a transfer of energy from the spin system to the external environment. Directly after a 90 degree pulse, energy has been transferred into the spin system and the longitudinal component ($\vec M_z$) is close to zero.  As the spin system loses this energy and returns to thermal equilibrium, the longitudinal component ($\vec M_z$) regrows (Fig.x). This energy transfer is not spontaneous, however. To add energy to the spin system, $\vec B_1$ must oscillate at the Larmor frequency. Similarly, energy emissions must be stimulated by a transverse magnetic field fluctuating near the Larmour frequency. These feild fluctuations occur due to molecular motion of nearby nuclei or electrons. The rate at which spins encounter fields fluctuating at the frequency needed for energy transfer is highly dependendent on the molecular structure of the material or tissue being measured, resulting in a wide range of longitudinal relaxation times across different materials. The time constant that describes the longitudinal relaxation time is T1. Fig.x shows this relaxation after a 90 pulse. 

$$M_z(t) = M_0 \cdot (1-e^{-t/T_1})$$


TODO: Something about T1 relaxation being field dependent. Higher fields require higher frequency motion to stimulate emission. Less likely to occur so relaxation time is usually longer at higher field. 

TODO: more complex nature of T1 relaxation when a 90 pulse occurs before equilibrium has been reached. 

### 1.2. Transverse (T2) Relaxation
Transverse relaxation is caused by the dephasing of the individual magnetic moments. As phase coherence is lost, the transverse component ($\vec M_{xy}$) returns to zero (Fig.x). Transverse relaxation occurs more rapidly than longitudinal relaxation. The energy emissions that cause longitudinal relaxation randomly change the direction and phase of a magnetic moment resulting in dephasing in the transverse plane. However, dephasing can occur without energy transfer into the environment. 

TODO: List the main ways that this can occur so students can research them later. Introduce T2* and talk about which dephasing affects are reversable and which are not. Introduce T2 time constant. Talk about how T2 is largely field independent.  

$$M_{xy}(t) = M_{xy}(0) \cdot e^{-t/T_2}$$

### 1.3. Contrast Imaging 
The relaxation properties can be used to create contrast between different components in an image. This contrast can be controlled by looking at how the relaxation properties interact with the adjustable parameters of the pulse sequences. What are the main types of pulse sequences that will be explored in this lab? Some sort of overview of what we are working with. 

## 2. TR

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2:**
> 1. Step one
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# import importlib
# import multi_dot_plot
# importlib.reload(multi_dot_plot)

from bokeh.palettes import Viridis
from ROI_stats_image_plot import ROIStatsImagePlot
from multi_dot_plot import MultiDotPlot
from matipo.util.plots import PLOT_COLORS

from matipo.util.decimation import decimate
from matipo.util.fft import fft_reconstruction
from matipo.util.etl import deinterlace

class VariableTRImageExperiment(BaseExperiment):
    seq = Sequence('TR_TE_RARE.py')
    enable_runloop=True
    enable_partialplot=True
    plots = {
        'image': ROIStatsImagePlot(figure_opts=dict(
            title="Spiders"),color_palette=PLOT_COLORS),
        'roi_history': MultiDotPlot(figure_opts=dict(
            title="Amplitude vs TR History",
            x_axis_label='Rep. Time (s)',
            y_axis_label='Amplitude')
            ,color_palette=PLOT_COLORS)
    }

    data_history = {
        'TR': [],
        'Amp_1': [],
        'Amp_2': [],
        'Amp_3': [],
        #'Amp_4': []
    }
    
    roi=dict(
        center_x=[0,2,5],
        center_y=[0,2,4],
        radius=[1,0.5,2]
    )
    
    DECIMATION = 4
    FOV = 15

    def update_par(self):
        #TODO calculate parameters for correct FOV
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'frequency.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_90.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_180.yaml'))
        self.seq.setpar(
            t_rep = t_rep.value,
            n_scans=2,

            n_ETL=16,

            # for 2D, just use phase_2 and set n_phase_1 to 1 with no gradient
            n_phase_1=1,
            g_phase_1=(0,0,0),

            # 64 phase steps
            n_phase_2=64,
            g_phase_2=(0, 1.0, 0),

            t_dw=4e-6,
            n_samples=64*self.DECIMATION,
            g_read=(0.6, 0, 0),
            
            enable_dummy_run=True
        )
        # set calculated read gradient pulse duration
        self.seq.setpar(t_read=self.seq.par.t_dw*self.seq.par.n_samples)
        # set phase  pulse duration so that the area is half the read pulse area
        self.seq.setpar(t_phase=np.abs(np.linalg.norm(self.seq.par.g_read)/np.linalg.norm(self.seq.par.g_phase_2))*self.seq.par.t_read/2)
        # set minimum t_echo
        self.seq.setpar(t_echo=self.seq.par.t_180 + 4*self.seq.par.t_grad_ramp + 2*self.seq.par.t_phase + self.seq.par.t_read + 2*1e-7)
        
    async def update_plots(self):
        data = await self.seq.fetch_data()
        kdata = decimate(
            deinterlace(data, self.seq.par.n_ETL, self.seq.par.n_phase_1, self.seq.par.n_phase_2, self.seq.par.n_samples), 
            self.DECIMATION, axis=1)
        self.plots['image'].update(kdata, self.FOV, self.roi['center_x'], self.roi['center_y'], self.roi['radius'])
        if self.progress.value == self.progress.max:
            self.data_history['TR'].append(self.seq.par.t_rep)
            self.data_history['Amp_1'].append(self.plots['image'].roi_mean[0])
            self.data_history['Amp_2'].append(self.plots['image'].roi_mean[1])
            self.data_history['Amp_3'].append(self.plots['image'].roi_mean[2])
            #self.data_history['Amp_4'].append(self.plots['image'].roi_mean[3])
            self.plots['roi_history'].update(dict(
                Amp_1 = dict(x=self.data_history['TR'],y= self.data_history['Amp_1']),
                Amp_2 = dict(x=self.data_history['TR'],y= self.data_history['Amp_2']),
                Amp_3 = dict(x=self.data_history['TR'],y= self.data_history['Amp_3']),
                #Amp_4 = dict(x=self.data_history['TR'],y= self.data_history['Amp_4'])
            ))
            
t_rep = pn.widgets.FloatInput(name='Rep. Time (s)', value=0.5, start=0.01, step=0.01, width=200)
exp1 = VariableTRImageExperiment()
pn.Column(t_rep,exp1(), sizing_mode="stretch_both").servable()

## 3. TE

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2:**
> 1. Step one
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# import importlib
# import multi_dot_plot
# importlib.reload(multi_dot_plot)

# from https://stackoverflow.com/a/70635601
def getDivs(N):
    factors = {1}
    maxP  = int(N**0.5)
    p,inc = 2,1
    while p <= maxP:
        while N%p==0:
            factors.update([f*p for f in factors])
            N //= p
            maxP = int(N**0.5)
        p,inc = p+inc,2
    if N>1:
        factors.update([f*N for f in factors])
    return sorted(factors)

class VariableTEImageExperiment(BaseExperiment):
    seq = Sequence('TR_TE_RARE.py')
    enable_runloop=True
    enable_partialplot=True
    plots = {
        'image': ROIStatsImagePlot(figure_opts=dict(
            title="Spiders"),color_palette=PLOT_COLORS),
        'roi_history': MultiDotPlot(figure_opts=dict(
            title="Amplitude vs TE History",
            x_axis_label='Echo Time (s)',
            y_axis_label='Amplitude')
            ,color_palette=PLOT_COLORS)
    }

    data_history = {
        'TE': [],
        'Amp_1': [],
        'Amp_2': [],
        'Amp_3': [],
        #'Amp_4': []
    }
    
    roi=dict(
        center_x=[0,2,5],
        center_y=[0,2,4],
        radius=[1,0.5,2]
    )
    
    DECIMATION = 4
    FOV = 15
    
    t_echo_eff_input = pn.widgets.FloatInput(name='Effective Echo Time (s)', value=100e-3, start=10e-3, end=0.5, step=10e-3, width=200)
    n_ETL_display = pn.widgets.FloatInput(name='Echo Train Length', value=60, width=200, disabled=True)

    def update_par(self):
        #TODO calculate parameters for correct FOV
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'frequency.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_90.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_180.yaml'))
        self.seq.setpar(
            t_rep = 1,
            n_scans=2,

            n_ETL=60,

            # for 2D, just use phase_2 and set n_phase_1 to 1 with no gradient
            n_phase_1=1,
            g_phase_1=(0,0,0),

            # 64 phase steps
            n_phase_2=60,
            g_phase_2=(0, 1.0, 0),

            t_dw=4e-6,
            n_samples=60*self.DECIMATION,
            g_read=(0.6, 0, 0),
            
            enable_dummy_run=False
        )
        # set calculated read gradient pulse duration
        self.seq.setpar(t_read=self.seq.par.t_dw*self.seq.par.n_samples)
        # set phase  pulse duration so that the area is half the read pulse area
        self.seq.setpar(t_phase=np.abs(np.linalg.norm(self.seq.par.g_read)/np.linalg.norm(self.seq.par.g_phase_2))*self.seq.par.t_read/2)
        
        # calculate min possible t_echo
        t_echo_min = self.seq.par.t_180 + 4*self.seq.par.t_grad_ramp + 2*self.seq.par.t_phase + self.seq.par.t_read + 2*1e-7
        self.t_echo_eff = self.t_echo_eff_input.value
        # try good ETL options (divisors of the number of phase steps) until it works, starting with largest
        for n_ETL in reversed(getDivs(self.seq.par.n_phase_2)):
            t_echo = self.t_echo_eff/(n_ETL/2 + 1)
            if t_echo >= t_echo_min:
                break
        else:
            raise Exception(f't_echo_eff={self.t_echo_eff} too short!')
        self.n_ETL_display.value = n_ETL
        self.seq.setpar(n_ETL=n_ETL, t_echo=t_echo)
         
        
    async def update_plots(self):
        data = await self.seq.fetch_data()
        kdata = decimate(
            deinterlace(data, self.seq.par.n_ETL, self.seq.par.n_phase_1, self.seq.par.n_phase_2, self.seq.par.n_samples), 
            self.DECIMATION, axis=1)
        self.plots['image'].update(kdata, self.FOV, self.roi['center_x'], self.roi['center_y'], self.roi['radius'])
        if self.progress.value == self.progress.max:
            self.data_history['TE'].append(self.t_echo_eff)
            self.data_history['Amp_1'].append(self.plots['image'].roi_mean[0])
            self.data_history['Amp_2'].append(self.plots['image'].roi_mean[1])
            self.data_history['Amp_3'].append(self.plots['image'].roi_mean[2])
            self.plots['roi_history'].update(dict(
                Amp_1 = dict(x=self.data_history['TE'],y= self.data_history['Amp_1']),
                Amp_2 = dict(x=self.data_history['TE'],y= self.data_history['Amp_2']),
                Amp_3 = dict(x=self.data_history['TE'],y= self.data_history['Amp_3']),
            ))

exp2 = VariableTEImageExperiment()
# exp2.t_echo_eff = pn.widgets.FloatInput(name='Effective Echo Time (s)', value=100e-3, start=1e-3, end=0.5, step=10e-3, width=200)
# exp2.n_ETL_display = pn.widgets.FloatInput(name='Echo Train Length', value=60, width=200, disabled=True)

pn.Column(
    pn.Row(exp2.t_echo_eff_input, exp2.n_ETL_display),
    exp2(),
    sizing_mode="stretch_both"
).servable()

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Task 2:**
> 1. Step one
> -------------------------------------------------------------------------------------------------------------------------------------------------------

## 4. Contrast Imaging

In [ ]:
# import importlib
# import multi_dot_plot
# importlib.reload(multi_dot_plot)

class ContrastExperiment(BaseExperiment):
    seq = Sequence('TR_TE_RARE.py')
    enable_runloop=True
    enable_partialplot=True
    plots = {
        'image': ROIStatsImagePlot(figure_opts=dict(
            title="Spiders"),color_palette=PLOT_COLORS),
    }
    
    roi=dict(
        center_x=[],
        center_y=[],
        radius=[]
    )
    
    DECIMATION = 4
    FOV = 15
    
    t_rep_input = pn.widgets.FloatInput(name='Rep. Time (s)', value=2, start=0.01, step=0.1, width=200) 
    t_echo_eff_input = pn.widgets.FloatInput(name='Effective Echo Time (s)', value=100e-3, start=10e-3, end=0.5, step=10e-3, width=200)
    n_ETL_display = pn.widgets.FloatInput(name='Echo Train Length', value=60, width=200, disabled=True)

    def update_par(self):
        #TODO calculate parameters for correct FOV
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'frequency.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_90.yaml'))
        self.seq.loadpar(os.path.join(GLOBALS_DIR, 'hardpulse_180.yaml'))
        self.seq.setpar(
            t_rep = self.t_rep_input.value,
            n_scans=2,

            n_ETL=60,

            # for 2D, just use phase_2 and set n_phase_1 to 1 with no gradient
            n_phase_1=1,
            g_phase_1=(0,0,0),

            # 60 phase steps
            n_phase_2=60,
            g_phase_2=(0, 1.0, 0),

            t_dw=4e-6,
            n_samples=60*self.DECIMATION,
            g_read=(0.6, 0, 0),
            
            enable_dummy_run=True
        )
        # set calculated read gradient pulse duration
        self.seq.setpar(t_read=self.seq.par.t_dw*self.seq.par.n_samples)
        # set phase  pulse duration so that the area is half the read pulse area
        self.seq.setpar(t_phase=np.abs(np.linalg.norm(self.seq.par.g_read)/np.linalg.norm(self.seq.par.g_phase_2))*self.seq.par.t_read/2)
        
        # calculate min possible t_echo
        t_echo_min = self.seq.par.t_180 + 4*self.seq.par.t_grad_ramp + 2*self.seq.par.t_phase + self.seq.par.t_read + 2*1e-7
        self.t_echo_eff = self.t_echo_eff_input.value
        # try good ETL options (divisors of the number of phase steps) until it works, starting with largest
        for n_ETL in reversed(getDivs(self.seq.par.n_phase_2)):
            t_echo = self.t_echo_eff/(n_ETL/2 + 1)
            if t_echo >= t_echo_min:
                break
        else:
            raise Exception(f't_echo_eff={self.t_echo_eff} too short!')
        self.n_ETL_display.value = n_ETL
        self.seq.setpar(n_ETL=n_ETL, t_echo=t_echo)
        
    async def update_plots(self):
        data = await self.seq.fetch_data()
        kdata = decimate(
            deinterlace(data, self.seq.par.n_ETL, self.seq.par.n_phase_1, self.seq.par.n_phase_2, self.seq.par.n_samples), 
            self.DECIMATION, axis=1)
        self.plots['image'].update(kdata, self.FOV, self.roi['center_x'], self.roi['center_y'], self.roi['radius'])
        
# t_rep = pn.widgets.FloatInput(name='Rep. Time (s)', value=2, start=0.01, step=0.1, width=200)          
# t_echo_eff = pn.widgets.FloatInput(name='Effective Echo Time (s)', value=30e-3, start=20e-3, step=10e-3, width=200)
exp3 = ContrastExperiment()
pn.Column(
    pn.Row(exp3.t_rep_input, exp3.t_echo_eff_input, exp3.n_ETL_display),
    exp3(),
    sizing_mode="stretch_both"
).servable()